# Predictor de Popularidade de Músicas

**Índice**<a id='toc0_'></a>    
1. [Introdução](#toc1_)    
2. [Preparação do notebook para R e Python em sumultâneo](#toc2_)    
3. [Importação dos módulos](#toc3_)    
4. [Exploração dos dados em bruto](#toc4_)    
4.1. [Upload dos dados](#toc4_1_)    
4.2. [Criação do dataframe](#toc4_2_)    
4.3. [Visualização do dataframe em bruto e de algumas métricas](#toc4_3_)    
5. [Análise Exploratória](#toc5_)    
5.1. [Distribuição dos valores em cada coluna](#toc5_1_)    
6. [Pré-processamento dos dados](#toc6_)    
6.1. [Identificação de Valores Ausentes](#toc6_1_)    
6.2. [Remoção de musicas com tempo 0](#toc6_2_)    
6.3. [Identificação e remoção de duplicados](#toc6_3_)    
6.4. [Definir o índice](#toc6_4_)    
6.5. [Remover coluna song_name](#toc6_5_)    
6.6. [Definir variáveis categóricas e variáveis numéricas](#toc6_6_)    
6.7. [Método do intervalo interquartil (IQR) para filtrar outliers](#toc6_7_)    
6.8. [Remoção de outliers com o z-score](#toc6_8_)    
6.9. [Remoção de outliers com Local Outlier Factor (LOF)](#toc6_9_)    
6.10. [Remoção de outliers variável a variável](#toc6_10_)    
6.11. [Gráficos comparativos da remoção dos outliers com os diferentes métodos](#toc6_11_)    
6.12. [Inferência](#toc6_12_)    
6.13. [Estatísticas do dataset](#toc6_13_)    
6.14. [Divisão das variáveis dependentes e independentes](#toc6_14_)    
7. [Divisão em conjunto de treino e conjunto de teste](#toc7_)    
8. [Implementação de algoritmos de Aprendizagem Computacional](#toc8_)    
8.1. [Random Forest](#toc8_1_)    
8.1.1. [Treino inicial do modelo](#toc8_1_1_)    
8.1.2. [Matriz de confusão](#toc8_1_2_)    
8.1.3. [Afinação dos hiperparâmetros com o Grid Search Cross Validation](#toc8_1_3_)    
8.1.4. [Configurar e executar o Grid Search Cross Validation](#toc8_1_4_)    
8.1.5. [Treino do modelo com os melhores hiperparâmetros](#toc8_1_5_)    
9. [Resultados](#toc9_)    
10. [Considerações finais](#toc10_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Introdução](#toc0_)

## 2. <a id='toc2_'></a>[Preparação do notebook para R e Python em sumultâneo](#toc0_)

In [ ]:
# ! R -e "install.packages('IRkernel', repos = 'http://cran.us.r-project.org');IRkernel::installspec()"

In [ ]:
# ! pip install rpy2

In [ ]:
%load_ext rpy2.ipython

In [ ]:
#%%R -i df_no_outliers
#head(df_no_outliers) #Exemplo de utilização

## 3. <a id='toc3_'></a>[Importação dos módulos](#toc0_)

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt


## 4. <a id='toc4_'></a>[Exploração dos dados em bruto](#toc0_)

### 4.1. <a id='toc4_1_'></a>[Upload dos dados](#toc0_)

In [ ]:
data = pd.read_csv('data/song_data.csv')

### 4.2. <a id='toc4_2_'></a>[Criação do dataframe](#toc0_)

In [ ]:
df_raw = pd.DataFrame(data)

In [ ]:
df_raw

### 4.3. <a id='toc4_3_'></a>[Visualização do dataframe em bruto e de algumas métricas](#toc0_)

In [ ]:
df_raw.info()

In [ ]:
df_raw.shape

## 5. <a id='toc5_'></a>[Análise Exploratória](#toc0_)

### 5.1. <a id='toc5_1_'></a>[Distribuição dos valores em cada coluna](#toc0_)

In [ ]:
col_names = df_raw.columns

col_names

In [ ]:
col_counts = {}

for col in col_names:
    col_counts[col] = df_raw[col].value_counts()

col_counts


## 6. <a id='toc6_'></a>[Pré-processamento dos dados](#toc0_)

### 6.1. <a id='toc6_1_'></a>[Identificação de Valores Ausentes](#toc0_)

In [ ]:
df_raw.isnull().sum()

### 6.2. <a id='toc6_2_'></a>[Remoção de musicas com tempo 0](#toc0_)

In [ ]:
df_raw.drop(df_raw[(df_raw['tempo']==0)].index, inplace=True)

### 6.3. <a id='toc6_3_'></a>[Identificação e remoção de duplicados](#toc0_)

In [ ]:
# Remover duplicatas considerando todas as colunas
df_raw.drop_duplicates(inplace=True)


In [ ]:
df_raw.shape

### 6.4. <a id='toc6_4_'></a>[Definir o índice](#toc0_)

In [ ]:
df_raw.set_index(pd.RangeIndex(start=1, stop=len(df_raw)+1, step=1), inplace=True)

### 6.5. <a id='toc6_5_'></a>[Remover coluna song_name](#toc0_)

In [ ]:
df_raw.drop(columns='song_name', inplace=True)


### 6.6. <a id='toc6_6_'></a>[Definir variáveis categóricas e variáveis numéricas](#toc0_)

In [ ]:
from sklearn.compose import make_column_selector as selector
numerical_columns_selector = selector(dtype_exclude='category')
categorical_columns_selector = selector(dtype_include='category')

numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(df_raw) 
df_raw.loc[:,:] = scaled_values

### 6.7. <a id='toc6_7_'></a>[Método do intervalo interquartil (IQR) para filtrar outliers](#toc0_)

In [ ]:
numeric_columns = df_raw.select_dtypes(include='number')

# Calcula Q1, Q3 e IQR para cada coluna numérica
Q1 = numeric_columns.quantile(0.25)
Q3 = numeric_columns.quantile(0.75)
IQR = Q3 - Q1

# Define os limites para identificar outliers
lower_bound = Q1 - 1.5 * IQR 
upper_bound = Q3 + 1.5 * IQR

# Identifica e remove outliers
outliers = ((numeric_columns < lower_bound) | (numeric_columns > upper_bound)).any(axis=1)
df_IQR = df_raw[~outliers]
df_IQR.shape

### 6.8. <a id='toc6_8_'></a>[Remoção de outliers com o z-score](#toc0_)

In [ ]:
# Calcular os z-scores para todas as colunas
z_scores = np.abs(stats.zscore(df_raw))

# Definir um threshold para outliers (por exemplo, Z-score < 3)
threshold = 3

# Manter as linhas onde os z-scores são menores que o threshold para todas as colunas
df_Z = df_raw[(z_scores < threshold).all(axis=1)]

### 6.9. <a id='toc6_9_'></a>[Remoção de outliers com Local Outlier Factor (LOF)](#toc0_)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor

# Supondo que df_raw é o seu DataFrame carregado com várias colunas
# Se df_raw não estiver definido, você pode carregá-lo assim:
# df_raw = pd.read_csv('caminho/para/seu/arquivo.csv')

# Load the dataset
X = df_raw

# Create the LocalOutlierFactor model for outlier detection
lof_outlier = LocalOutlierFactor(n_neighbors=20)

# Fit the model to the data and predict the outlier scores for each data point
outlier_scores = lof_outlier.fit_predict(X)

# Identify the outlier data points
outlier_indices = outlier_scores == -1
print("Outlier indices:", outlier_indices)

# Remover os outliers do DataFrame
df_LOF = X[~outlier_indices]
df_LOF.shape

### 6.10. <a id='toc6_10_'></a>[Remoção de outliers variável a variável](#toc0_)

### 6.11. <a id='toc6_11_'></a>[Gráficos comparativos da remoção dos outliers com os diferentes métodos](#toc0_)

In [ ]:
# Plot histograms and boxplots for each variable in a single image
variables = X.columns
num_vars = len(variables)

fig, axes = plt.subplots(num_vars, 2, figsize=(14, 5*num_vars))

for i, variable in enumerate(variables):
    # Histogram with and without outliers
    sns.histplot(X[variable], bins=50, kde=True, color='blue', ax=axes[i, 0], label='Original')
    sns.histplot(df_LOF[variable], bins=50, kde=True, color='orange', ax=axes[i, 0], label='LOF')
    sns.histplot(df_Z[variable], bins=50, kde=True, color='green', ax=axes[i, 0], label='Z-score')
    sns.histplot(df_IQR[variable], bins=50, kde=True, color='red', ax=axes[i, 0], label='IRQ')
    axes[i, 0].set_title(f'{variable} - Histogram')
    axes[i, 0].legend()

    # Boxplot with and without outliers
    sns.boxplot(data=[X[variable], df_LOF[variable]], ax=axes[i, 1])
    axes[i, 1].set_xticklabels(['With Outliers', 'Without Outliers'])
    axes[i, 1].set_title(f'{variable} - Boxplot')

plt.tight_layout()
plt.show()


In [ ]:
np.random.seed(2022)
x = np.random.randn(100)

f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)})

sns.boxplot(x=x, ax=ax_box)
sns.histplot(x=x, bins=12, kde=True, stat='density', ax=ax_hist)

ax_box.set(yticks=[])
sns.despine(ax=ax_hist)
sns.despine(ax=ax_box, left=True)

### 6.12. <a id='toc6_12_'></a>[Inferência](#toc0_)

Conjunto de dados inicial: 18835 amostras.

Com o método do Intervalo Interquartil: 

Com o método do Z-score:

Com o método do Local Outlier Factor:


### 6.13. <a id='toc6_13_'></a>[Estatísticas do dataset](#toc0_)

In [ ]:
import matplotlib.pyplot as plt

# Definindo as cores
colors = ['green', 'red']  # Exemplo de cores: azul e laranja

fig, axes = plt.subplots(1, 3, figsize=(15, 5))  # Criando uma figura com 3 subplots

# Gráfico para df_IQR
axes[0].set_title('Método IQR')
axes[0].pie([df_IQR.shape[0], df_raw.shape[0] - df_IQR.shape[0]],
            radius=1,
            labels=['Mantido', 'Excluído'],
            counterclock=False,
            autopct='%1.1f%%',
            pctdistance=0.9,
            explode=[0.02, 0],
            shadow=False,
            colors=colors)  # Adicionando a lista de cores

# Gráfico para df_Z
axes[1].set_title('Método Z-score')
axes[1].pie([df_Z.shape[0], df_raw.shape[0] - df_Z.shape[0]],
            radius=1,
            labels=['Mantido', 'Excluído'],
            counterclock=False,
            autopct='%1.1f%%',
            pctdistance=0.9,
            explode=[0.02, 0],
            shadow=False,
            colors=colors)  # Adicionando a lista de cores

# Gráfico para df_LOF
axes[2].set_title('Método LOF')
axes[2].pie([df_LOF.shape[0], df_raw.shape[0] - df_LOF.shape[0]],
            radius=1,
            labels=['Mantido', 'Excluído'],
            counterclock=False,
            autopct='%1.1f%%',
            pctdistance=0.9,
            explode=[0.02, 0],
            shadow=False,
            colors=colors)  # Adicionando a lista de cores

plt.tight_layout()  # Ajusta o layout para evitar sobreposição de legendas
plt.show()



### 6.14. <a id='toc6_14_'></a>[Divisão das variáveis dependentes e independentes](#toc0_)

In [ ]:
# Splitting the data intro training & testing sets

from sklearn.model_selection import train_test_split

#m=[]
#for i in df_IQR.columns.values:
#    m.append(i.replace(' ','_'))
#    
#df_IQR.columns = m
#X = df_IQR[['song_duration_ms', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode', 'speechiness', #'tempo', 'time_signature', 'audio_valence']]
#X.loc[:, ['key', 'audio_mode', 'time_signature']] = X.loc[:, ['key', 'audio_mode', 'time_signature']].astype('category')
#Y = df_IQR["song_popularity"]



#m=[]
#for i in df_Z.columns.values:
#    m.append(i.replace(' ','_'))
#    
#df_Z.columns = m
#X = df_Z[['song_duration_ms', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode', 'speechiness', 'tempo', #'time_signature', 'audio_valence']]
#X.loc[:, ['key', 'audio_mode', 'time_signature']] = X.loc[:, ['key', 'audio_mode', 'time_signature']].astype('category')
#Y = df_Z["song_popularity"]




m=[]
for i in df_LOF.columns.values:
    m.append(i.replace(' ','_'))
    
df_LOF.columns = m
X = df_LOF[['song_duration_ms', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode', 'speechiness', 'tempo', 'time_signature', 'audio_valence']]
X.loc[:, ['key', 'audio_mode', 'time_signature']] = X.loc[:, ['key', 'audio_mode', 'time_signature']].astype('category')
Y = df_LOF["song_popularity"]



## 7. <a id='toc7_'></a>[Divisão em conjunto de treino e conjunto de teste](#toc0_)

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=42)
Train_X.reset_index(drop=True,inplace=True)

print('Conjunto original (100%): ',X.shape,Y.shape,'\nConjunto de treino (80%): ',Train_X.shape,Train_Y.shape,'\nConjunto de teste (20%): ', Test_X.shape,'', Test_Y.shape)

## 8. <a id='toc8_'></a>[Implementação de algoritmos de Aprendizagem Computacional](#toc0_)

### 8.1. <a id='toc8_1_'></a>[Random Forest](#toc0_)

#### 8.1.1. <a id='toc8_1_1_'></a>[Treino inicial do modelo](#toc0_)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn import set_config

# Escolha RandomForestClassifier para problemas de classificação e RandomForestRegressor para regressão
model = make_pipeline(RandomForestRegressor(random_state=21))  # ou RandomForestRegressor
set_config(display='diagram')

# Treinar o modelo
model.fit(Train_X, Train_Y)

# Fazer previsões
Pred_Y = model.predict(Test_X)
model



In [ ]:
def modelresults(predictions):
    mae = mean_absolute_error(Test_Y, predictions)
    mse = mean_squared_error(Test_Y, predictions)
    r2 = r2_score(Test_Y, predictions)
    
    print('Erro absoluto médio do modelo: {:.4f}'.format(mae))
    print('')
    print('Erro quadrático médio do modelo: {:.4f}'.format(mse))
    print('')
    print('O valor de r2 do modelo: {:.4f}'.format(r2))

In [ ]:
modelresults(Pred_Y)

#### 8.1.2. <a id='toc8_1_2_'></a>[Matriz de confusão](#toc0_)

#### 8.1.3. <a id='toc8_1_3_'></a>[Afinação dos hiperparâmetros com o Grid Search Cross Validation](#toc0_)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],      # Número de árvores in the forest
    'max_features': ['sqrt', 'log2'],     # Número de features a serem consideradas para divisão
    'max_depth': [None, 10, 20, 30],      # Profundidade máxima da árvore
    'min_samples_split': [2, 5, 10],      # Número mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4]         # Número mínimo de amostras num nó folha
}


#### 8.1.4. <a id='toc8_1_4_'></a>[Configurar e executar o Grid Search Cross Validation](#toc0_)

In [ ]:
# Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=21), param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Executar o GridSearchCV
grid_search.fit(Train_X, Train_Y)

# Obter os melhores hiperparâmetros
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_
print(f"Melhor estimador: {best_estimator}")
print(f'Melhores hiperparâmetros: {best_params}')


#### 8.1.5. <a id='toc8_1_5_'></a>[Treino do modelo com os melhores hiperparâmetros](#toc0_)

In [ ]:
# Criar um novo modelo com os melhores hiperparâmetros
#best_model = RandomForestClassifier(**best_params, random_state=42)  # ou RandomForestRegressor
best_model = RandomForestRegressor(**best_params, random_state=42)  # ou RandomForestRegressor

# Treinar o modelo
best_model.fit(Train_X, Train_Y)

# Fazer previsões
Best_Pred_Y = best_model.predict(Test_X)


In [ ]:
modelresults(Best_Pred_Y)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


cm = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(Best_Pred_Y, Pred_Y),
                              display_labels=df_LOF.columns) 
cm.plot();

## 9. <a id='toc9_'></a>[Resultados](#toc0_)

## 10. <a id='toc10_'></a>[Considerações finais](#toc0_)